In [78]:
import csv
import sys
from collections import namedtuple
from itertools import islice


class ContextManagerIterator():
    def __init__(self, fname):
        self.fname = fname
        self._f = None
        self._dialect = None
        self._headers = None

    def __iter__(self):
        return self

    def __next__(self):
        return next(self._f)

    def __enter__(self):
        self._f = open(self.fname, 'r')
        return self

    def __exit__(self, exc_type, exc_value, exc_traceback):
        if not self._f.closed:
            self._f.close()
        return False

    @property
    def dialect(self):
        self._dialect = csv.Sniffer().sniff(self._f.read(1024))
        return self._dialect

    @property
    def headers(self):
        self._f.seek(0)
        reader = csv.reader(self._f, self._dialect)
        self._headers = next(reader)
        return self._headers

        # dialect = csv.Sniffer().sniff(self._f.read(1024))
    # reader = csv.reader(self._f, dialect)
    # print(next(reader))
    # Data = namedtuple('Data', next(reader))
    # for row in reader:
    #     return Data(*row)



In [79]:
with ContextManagerIterator('cars.csv') as data:
    reader = csv.reader(data, data.dialect)
    Data = namedtuple('Data', data.headers)
    for row in islice(reader, 3):
        print(Data(*row))

Data(Car='Chevrolet Chevelle Malibu', MPG='18.0', Cylinders='8', Displacement='307.0', Horsepower='130.0', Weight='3504.', Acceleration='12.0', Model='70', Origin='US')
Data(Car='Buick Skylark 320', MPG='15.0', Cylinders='8', Displacement='350.0', Horsepower='165.0', Weight='3693.', Acceleration='11.5', Model='70', Origin='US')
Data(Car='Plymouth Satellite', MPG='18.0', Cylinders='8', Displacement='318.0', Horsepower='150.0', Weight='3436.', Acceleration='11.0', Model='70', Origin='US')


In [80]:
with ContextManagerIterator('personal_info.csv') as data:
    reader = csv.reader(data, data.dialect)
    Data = namedtuple('Data', data.headers)
    for row in islice(reader, 3):
        print(Data(*row))

Data(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
Data(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
Data(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')


In [121]:
from contextlib import contextmanager


@contextmanager
def open_file(fname):
    print(f'opening file {fname}')
    f = open(fname)
    try:
        yield f
    finally:
        print(f'closing file {fname}')
        f.close()


In [136]:
with open_file('cars.csv') as f:
    dialect = csv.Sniffer().sniff(f.read(1024))
    f.seek(0)
    reader = csv.reader(f, dialect)
    headers = next(reader)
    Data = namedtuple('Data', headers)
    for row in islice(reader, 3):
        print(Data(*row))

opening file cars.csv
Data(Car='Chevrolet Chevelle Malibu', MPG='18.0', Cylinders='8', Displacement='307.0', Horsepower='130.0', Weight='3504.', Acceleration='12.0', Model='70', Origin='US')
Data(Car='Buick Skylark 320', MPG='15.0', Cylinders='8', Displacement='350.0', Horsepower='165.0', Weight='3693.', Acceleration='11.5', Model='70', Origin='US')
Data(Car='Plymouth Satellite', MPG='18.0', Cylinders='8', Displacement='318.0', Horsepower='150.0', Weight='3436.', Acceleration='11.0', Model='70', Origin='US')
closing file cars.csv


In [123]:
with open_file('personal_info.csv') as f:
    dialect = csv.Sniffer().sniff(f.read(1024))
    f.seek(0)
    reader = csv.reader(f, dialect)
    headers = next(reader)
    Data = namedtuple('Data', headers)
    for row in islice(reader, 3):
        print(Data(*row))

opening file personal_info.csv
Data(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
Data(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
Data(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')
closing file personal_info.csv


In [134]:
class ContextManagerIterator():
    def __init__(self, fname):
        self.fname = fname
        self._f = None
        self._dialect = None
        self._headers = None

    def __iter__(self):
        return self

    def __next__(self):
        return self._data(*next(self._reader))

    def __enter__(self):
        self._f = open(self.fname, 'r')
        self._reader = csv.reader(self._f, self.dialect)
        self._data = namedtuple('Data', self.headers)
        return self

    def __exit__(self, exc_type, exc_value, exc_traceback):
        if not self._f.closed:
            self._f.close()
        return False

    @property
    def dialect(self):
        self._dialect = csv.Sniffer().sniff(self._f.read(1024))
        return self._dialect

    @property
    def headers(self):
        self._f.seek(0)
        reader = csv.reader(self._f, self._dialect)
        self._headers = next(reader)
        return self._headers

        # dialect = csv.Sniffer().sniff(self._f.read(1024))
    # reader = csv.reader(self._f, dialect)
    # print(next(reader))
    # Data = namedtuple('Data', next(reader))
    # for row in reader:
    #     return Data(*row)


with ContextManagerIterator('cars.csv') as data:
    # reader = csv.reader(data, data.dialect)
    # Data = namedtuple('Data', data.headers)
    for row in islice(data, 3):
        print(row)

print('-----------------------------------------')

with ContextManagerIterator('personal_info.csv') as data:
    for row in islice(data, 3):
        print(row)

Data(Car='Chevrolet Chevelle Malibu', MPG='18.0', Cylinders='8', Displacement='307.0', Horsepower='130.0', Weight='3504.', Acceleration='12.0', Model='70', Origin='US')
Data(Car='Buick Skylark 320', MPG='15.0', Cylinders='8', Displacement='350.0', Horsepower='165.0', Weight='3693.', Acceleration='11.5', Model='70', Origin='US')
Data(Car='Plymouth Satellite', MPG='18.0', Cylinders='8', Displacement='318.0', Horsepower='150.0', Weight='3436.', Acceleration='11.0', Model='70', Origin='US')
-----------------------------------------
Data(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
Data(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
Data(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')


In [147]:
def parser_data_iter(data_iter, nt):
    for row in data_iter:
        yield nt(*row)


def get_dialect(fname):
    with open(fname) as f:
        return csv.Sniffer().sniff(f.read(1024))


@contextmanager
def open_file_ctx(fname):
    print(f'opening file {fname}')
    f = open(fname, 'r')
    try:
        reader = csv.reader(f, get_dialect(fname))
        headers = next(reader)
        Data = namedtuple('Data', headers)
        yield parser_data_iter(reader, Data)
    finally:
        print(f'closing file {fname}')
        f.close()


with open_file_ctx('cars.csv') as f:
    for row in islice(f, 3):
        print(row)
print('-----------------------------')
with open_file_ctx('personal_info.csv') as f:
    for row in islice(f, 3):
        print(row)

opening file cars.csv
Data(Car='Chevrolet Chevelle Malibu', MPG='18.0', Cylinders='8', Displacement='307.0', Horsepower='130.0', Weight='3504.', Acceleration='12.0', Model='70', Origin='US')
Data(Car='Buick Skylark 320', MPG='15.0', Cylinders='8', Displacement='350.0', Horsepower='165.0', Weight='3693.', Acceleration='11.5', Model='70', Origin='US')
Data(Car='Plymouth Satellite', MPG='18.0', Cylinders='8', Displacement='318.0', Horsepower='150.0', Weight='3436.', Acceleration='11.0', Model='70', Origin='US')
closing file cars.csv
-----------------------------
opening file personal_info.csv
Data(ssn='100-53-9824', first_name='Sebastiano', last_name='Tester', gender='Male', language='Icelandic')
Data(ssn='101-71-4702', first_name='Cayla', last_name='MacDonagh', gender='Female', language='Lao')
Data(ssn='101-84-0356', first_name='Nomi', last_name='Lipprose', gender='Female', language='Yiddish')
closing file personal_info.csv


In [152]:

@contextmanager
def open_file_ctx(fname):
    print(f'opening file {fname}')
    f = open(fname, 'r')
    try:
        dialect = csv.Sniffer().sniff(f.read(1024))
        f.seek(0)
        reader = csv.reader(f, dialect)
        headers = next(reader)
        Data = namedtuple('Data', headers)
        yield (Data(*row) for row in reader)
    finally:
        print(f'closing file {fname}')
        f.close()


with open_file_ctx('cars.csv') as f:
    for row in islice(f, 3):
        print(row)

opening file cars.csv
Data(Car='Chevrolet Chevelle Malibu', MPG='18.0', Cylinders='8', Displacement='307.0', Horsepower='130.0', Weight='3504.', Acceleration='12.0', Model='70', Origin='US')
Data(Car='Buick Skylark 320', MPG='15.0', Cylinders='8', Displacement='350.0', Horsepower='165.0', Weight='3693.', Acceleration='11.5', Model='70', Origin='US')
Data(Car='Plymouth Satellite', MPG='18.0', Cylinders='8', Displacement='318.0', Horsepower='150.0', Weight='3436.', Acceleration='11.0', Model='70', Origin='US')
closing file cars.csv
